In [ ]:
%load_ext Cython
import numpy as np
import pandas as pd
import pykonal

EARTH_RADIUS = 6371.
DTYPE_REAL = np.float64

In [ ]:
%%cython

import numpy as np
import scipy.optimize
cimport numpy as np


ctypedef np.float64_t _REAL_t

EARTH_RADIUS = 6371.
DTYPE_REAL = np.float64

def geo2sph(arr):
    """
    Map Geographical coordinates to spherical coordinates.
    """
    geo = np.array(arr, dtype=DTYPE_REAL)
    sph = np.empty_like(geo)
    sph[..., 0] = EARTH_RADIUS - geo[..., 2]
    sph[..., 1] = np.pi / 2 - np.radians(geo[..., 0])
    sph[..., 2] = np.radians(geo[..., 1])
    return (sph)


def sph2geo(arr):
    """
    Map spherical coordinates to geographic coordinates.
    """
    sph = np.array(arr, dtype=DTYPE_REAL)
    geo = np.empty_like(sph)
    geo[..., 0] = np.degrees(np.pi / 2 - sph[..., 1])
    geo[..., 1] = np.degrees(sph[..., 2])
    geo[..., 2] = EARTH_RADIUS - sph[..., 0]
    return (geo)


cdef class EQLocator(object):
    cdef dict _arrivals
    cdef dict _tt_calculators
    cdef _REAL_t[:,:,:,:] _grid
    cdef tuple _bounds
    cdef dict _priors
    
    def __init__(self, arrivals, tt_calculators, grid):
        self._arrivals = {key: arrivals[key] for key in tt_calculators}
        self._tt_calculators = tt_calculators
        self._grid = grid
    
    @property
    def arrivals(self):
        return (self._arrivals)
    
    @arrivals.setter
    def arrivals(self, value):
        self._arrivals = value
    
    @property
    def grid(self):
        return (np.asarray(self._grid))
    
    @grid.setter
    def grid(self, value):
        self._grid = value
        
    @property
    def tt_calculators(self):
        return (self._tt_calculators)
    
    @tt_calculators.setter
    def tt_calculators(self, value):
        self._tt_calculators = value
        
    cpdef initial_guess(self):
        values = [self.arrivals[key]-self.tt_calculators[key].values for key in self.tt_calculators]
        values = np.stack(values)
        std = values.std(axis=0)
        arg_min = np.argmin(std)
        idx_min = np.unravel_index(arg_min, std.shape)
        geo = sph2geo(self.grid[idx_min])
        time = values.mean(axis=0)[idx_min]
        return (np.array([*geo, time]))

    cpdef cost(self, _REAL_t[:] hypocenter):
        cdef tuple key
        cdef _REAL_t csum, lat, lon, depth, time
        lat = hypocenter[0]
        lon = hypocenter[1]
        depth = hypocenter[2]
        time = hypocenter[3]
        for key in self.arrivals:
            sph_coords = geo2sph((lat, lon, depth, time))
            tt_calculator = self.tt_calculators[key]
            csum += np.square(self.arrivals[key]-(time+tt_calculator(sph_coords)))
        return (np.sqrt(csum/len(self.arrivals)))
    
    cpdef locate(self):
        cdef _REAL_t[4] h0
        h0 = self.initial_guess()
        soln = scipy.optimize.differential_evolution(
            self.cost,
            ((h0[0]-0.25, h0[0]+0.25), (h0[1]-0.25, h0[1]+0.25), (0, 30), (h0[3]-5, h0[3]+5))
        )
        return (soln.x)

# Load a test data set

In [ ]:
class VelocityModel(object):
    def __init__(self, velocity, grid):
        self.velocity = velocity
        self.grid = grid
        
def geo2sph(arr):
    """
    Map Geographical coordinates to spherical coordinates.
    """
    geo = np.array(arr, dtype=DTYPE_REAL)
    sph = np.empty_like(geo)
    sph[..., 0] = EARTH_RADIUS - geo[..., 2]
    sph[..., 1] = np.pi / 2 - np.radians(geo[..., 0])
    sph[..., 2] = np.radians(geo[..., 1])
    return (sph)
        
def init_farfield(vmodel):
    """
    Initialize the far-field EikonalSolver with the given velocity model.
    """
    far_field = pykonal.EikonalSolver(coord_sys='spherical')
    far_field.vgrid.min_coords = vmodel.grid.min_coords
    far_field.vgrid.node_intervals = vmodel.grid.node_intervals
    far_field.vgrid.npts = vmodel.grid.npts
    far_field.vv = vmodel.velocity
    return (far_field)


def init_nearfield(far_field, origin):
    """
    Initialize the near-field EikonalSolver.
    :param origin: Station location in spherical coordinates.
    :type origin: (float, float, float)
    :return: Near-field EikonalSolver
    :rtype: pykonal.EikonalSolver
    """
    drho = far_field.vgrid.node_intervals[0] / 5
    near_field = pykonal.EikonalSolver(coord_sys='spherical')
    near_field.vgrid.min_coords = drho, 0, 0
    near_field.vgrid.node_intervals = drho, np.pi / 20, np.pi / 20
    near_field.vgrid.npts = 100, 21, 40
    near_field.transfer_velocity_from(far_field, origin)
    vvi = pykonal.LinearInterpolator3D(near_field.vgrid, near_field.vv)

    for it in range(near_field.pgrid.npts[1]):
        for ip in range(near_field.pgrid.npts[2]):
            idx = (0, it, ip)
            near_field.uu[idx] = near_field.pgrid[idx + (0,)] / vvi(near_field.pgrid[idx])
            near_field.is_far[idx] = False
            near_field.close.push(*idx)
    return (near_field)

def compute_traveltime_lookup_table(geo_coords, vmodel):
    rho0, theta0, phi0 = geo2sph(geo_coords)
    far_field = init_farfield(vmodel)
    near_field = init_nearfield(far_field, (rho0, theta0, phi0))
    near_field.solve()
    far_field.transfer_travel_times_from(near_field, (-rho0, theta0, phi0), set_alive=True)
    far_field.solve()
    return (far_field)

In [ ]:
# Load event data
with pd.HDFStore("/home/malcolmw/google_drive/malcolm.white@usc.edu/data/events/scedc/h5/scedc_2000-2019.h5") as store:
    df_events = store["events"]
    df_arrivals = store["arrivals"].set_index("event_id")
    
with pd.HDFStore("/home/malcolmw/google_drive/malcolm.white@usc.edu/data/networks/scsn.h5") as store:
    df_stations = store["stations"].set_index("station_id")
    df_stations["depth"] = -df_stations["elev"]

# Load velocity model
with np.load("/home/malcolmw/google_drive/malcolm.white@usc.edu/proj/tomo_socal/data/scec_cvms.P.smooth.npz") as npz:
    grid = pykonal.Grid3D(coord_sys="spherical")
    grid.min_coords = npz["min_coords"]
    grid.node_intervals = npz["node_intervals"]
    grid.npts = npz["npts"]
    vp = VelocityModel(npz["vv"], grid)

with np.load("/home/malcolmw/google_drive/malcolm.white@usc.edu/proj/tomo_socal/data/scec_cvms.S.smooth.npz") as npz:
    grid = pykonal.Grid3D(coord_sys="spherical")
    grid.min_coords = npz["min_coords"]
    grid.node_intervals = npz["node_intervals"]
    grid.npts = npz["npts"]
    vs = VelocityModel(npz["vv"], grid)
    

In [ ]:
event_id = 38924903
arrivals = df_arrivals.loc[event_id].set_index(["station_id", "phase"]).to_dict()["time"]

In [ ]:
tt_calculators = dict()

for arrival_id in arrivals.keys():
    station_id, phase = arrival_id
    if station_id not in df_stations.index:
        print(f"No metadata for {station_id}")
        continue
    solver = compute_traveltime_lookup_table(
        df_stations.loc[station_id, ["lat", "lon", "depth"]].values, 
        vp if phase is "P" else vs
    )
    tt_calculators[arrival_id] = pykonal.LinearInterpolator3D(
        solver.pgrid,
        solver.uu
    )

In [ ]:
locator = EQLocator(
    arrivals=arrivals,
    tt_calculators=tt_calculators,
    grid=solver.pgrid.nodes
)
locator.locate()

In [ ]:
locator.cost(np.array([33.4861667, -116.414667,  4.97000000,  1572537117.73]))

In [ ]:
locator.cost(h0)

In [ ]:
import scipy.optimize

In [ ]:
h0

In [ ]:
%%time
soln = scipy.optimize.differential_evolution(
    locator.cost,
    ((h0[0]-0.25, h0[0]+0.25), (h0[1]-0.25, h0[1]+0.25), (0, 30), (h0[3]-5, h0[3]+5))
)

In [ ]:
locator.cost(soln.x)

In [ ]:
h0, values, idx = locator.initial_guess()

In [ ]:
df_events.set_index("event_id").loc[event_id, "time"]